In [1]:
!pip install -U weaviate-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 603.5/603.5 kB 17.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 19.6 MB/s  0:00:00
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.27.1
    Uninstalling pydantic_core-2.27.1:
      Successfully uninstalled pydantic_core-2.27.1
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.10.3
    Uninstalling pydantic-2.10.3:
      Successfully uninstalled pydantic-2.10.3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7/7 [weaviate-client]6/7 [weaviate-client]


In [5]:
# docker run -d \
#  --name weaviate \
#  -p 8080:8080 \
#  -e AUTHENTICATION_ANONYMOUS_ACCESS_ENABLED=true \
#  cr.weaviate.io/semitechnologies/weaviate:1.27.0

import weaviate
from weaviate.classes.config import Property, DataType

# 1. Connect to local Weaviate instance
# v4 automatically handles the gRPC connection on port 50051
with weaviate.connect_to_local() as client:
    
    # 2. Create a Collection (Schema)
    # We specify the properties we want to store
    if client.collections.exists("TechArticles"):
        client.collections.delete("TechArticles")

    articles = client.collections.create(
        name="TechArticles",
        properties=[
            Property(name="title", data_type=DataType.TEXT),
            Property(name="content", data_type=DataType.TEXT),
        ]
    )

    # 3. Insert Data with Vectors
    # In a real app, you'd use an embedding model to generate these 3D vectors
    articles.data.insert(
        properties={"title": "Docker Guide", "content": "How to use containers"},
        vector=[0.1, 0.9, 0.1] 
    )
    articles.data.insert(
        properties={"title": "Python Tips", "content": "Writing clean code"},
        vector=[0.8, 0.1, 0.2]
    )

    # 4. Perform a Vector Search
    # We search for something "near" our first vector
    response = articles.query.near_vector(
        near_vector=[0.1, 0.8, 0.2],
        limit=1
    )

    for obj in response.objects:
        print(f"Found: {obj.properties['title']} (ID: {obj.uuid})")

Found: Docker Guide (ID: e9a75852-97bb-49a8-8068-bbb1b99976ca)
